In [1]:
# Importing data paths
import sys
sys.path.insert(1, '/Users/anastasiakaravdina/code/podcasts-recommender-system')
from config import PODCASTS_DATABASE_PATH_RAW, USER_REVIEWS_DATABASE_PATH_RAW, PODCASTS_DATABASE_PATH_PROCESSED, USER_REVIEWS_DATABASE_PATH_PROCESSED
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
#from gensim import models
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
%matplotlib inline
#from gensim.models import FastText as ft
from IPython.display import Image

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anastasiakaravdina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anastasiakaravdina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anastasiakaravdina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anastasiakaravdina/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
PODCASTS_DATABASE_PATH_RAW, USER_REVIEWS_DATABASE_PATH_RAW

('/data/raw/podcasts.csv', '/data/raw/user_reviews.csv')

In [4]:
!ls

EDA.ipynb                           evaluation_miniLM_embeddings.ipynb
categorypair_sim_matrices.npy       recommendation model building.ipynb


In [5]:
# Loading data 
df_podcasts = pd.read_csv(PODCASTS_DATABASE_PATH_RAW,header=None)
df_users = pd.read_csv(USER_REVIEWS_DATABASE_PATH_RAW,header=None)

FileNotFoundError: [Errno 2] No such file or directory: '/data/raw/podcasts.csv'

In [ ]:
columns_pod = ['id','name','url','studio','category','episode_count','avg_rating','total_ratings','description']
columns_users = ['id','podcasts_id','username','review_title','review','rating','date']

df_podcasts.columns = columns_pod
df_users.columns = columns_users
df_users = df_users.drop('id', axis=1)

In [ ]:
df_podcasts.info()

In [ ]:
# % of Null values
percentage_of_null_values = df_podcasts.isnull().sum()/df_podcasts.shape[0] * 100

In [ ]:
df_null_distribution = pd.DataFrame(df_podcasts.isnull().sum()/df_podcasts.shape[0] * 100).reset_index()
df_null_distribution.columns = [['field in df_podcasts','% of null values']]

# Distribution of null values

In [ ]:
print('Distribution of null values')
df_null_distribution.style.background_gradient()

studio column has 61% of null values and also it won't be that useful for recommendations

# Top 10 Podcasts

In [ ]:
df_podcasts_top_10 = df_podcasts[df_podcasts['total_ratings'] > 70000][['name','category','episode_count','avg_rating','total_ratings']].reset_index()
df_podcasts_top_10 = df_podcasts_top_10.drop('index', axis=1)


In [ ]:
df = df_podcasts_top_10
fig = px.bar(df.sort_values(by='total_ratings',ascending=False), x='name', y='avg_rating',
             hover_data=['total_ratings', 'episode_count'], color='total_ratings',
             labels={'avg_rating':'average rating','name':'podcast'})
fig.update_layout(title='Top 10 Podcasts based on ratings')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()

# Top 10 podcasts with most episodes

In [ ]:
import plotly.express as px
df = df_podcasts
fig = px.histogram(df, x="episode_count",nbins=20,marginal="box",
                   labels={'episode_count':'No of episodes in a podcast'})
fig.update_layout(title='Distribution of no of episodes')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()


`The distribution is highly-skewed; the median number of episodes is 140 whereas the maximum number of episodes is around 2000.`

In [ ]:
df_category_avg_episode_count  = pd.DataFrame(df_podcasts.groupby(['category'])['episode_count'].mean().sort_values(ascending=False)).reset_index()
df_category_avg_episode_count.rename(columns={"episode_count": "avg_episode_count"},inplace=True)


In [ ]:
df = df_category_avg_episode_count
fig = px.bar(df, x='category', y='avg_episode_count', color='avg_episode_count',
             labels={'avg_episode_count':'average episode count'})
fig.update_layout(title='Category wise average no of episodes')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()

# Distribution of top 5 categories

In [ ]:
df_top_5_categories = df_podcasts[(df_podcasts['category'] == 'Business') | (df_podcasts['category'] == 'Sports') |
                (df_podcasts['category'] == 'News') | (df_podcasts['category'] == 'Science') |
                (df_podcasts['category'] == 'Leisure')]

In [ ]:
import plotly.express as px
df = df_top_5_categories
fig = px.histogram(df, x="episode_count",nbins=20,marginal="box",color='category',
                   labels={'episode_count':'No of episodes in a podcast'})
fig.update_layout(title='Distribution of no of episodes for categories :: News , Sports, Science, Business, Leisure')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()


# Ratings distribution

In [ ]:
df_category_avg_rating  = pd.DataFrame(df_podcasts.groupby(['category'])['avg_rating'].mean().sort_values(ascending=False)).reset_index()
df_category_avg_rating.rename(columns={"avg_rating": "avg_rating_per_category"},inplace=True)


In [ ]:
df = df_category_avg_rating
fig = px.bar(df, x='category', y='avg_rating_per_category', color='avg_rating_per_category',
             labels={'avg_rating_per_category':'average rating per category'})
fig.update_layout(title='Category wise average rating')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()

In [ ]:
import plotly.express as px
df = df_podcasts
fig = px.histogram(df, x="avg_rating",nbins=20,marginal="box",
                   labels={'episode_count':'No of episodes in a podcast'})
fig.update_layout(title='Distribution of average ratings')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()

In [ ]:
df_top_5_categories = df_podcasts[(df_podcasts['category'] == 'Religion & Spirituality') | (df_podcasts['category'] == 'Business') |
                (df_podcasts['category'] == 'Education') | (df_podcasts['category'] == 'Comedy') |
                (df_podcasts['category'] == 'Health & Fitness')]

In [ ]:
import plotly.express as px
df = df_top_5_categories
fig = px.histogram(df, x="avg_rating",nbins=20,marginal="box",color='category',
                   labels={'avg_rating':'average rating per category'})
fig.update_layout(title='Distribution of average rating for categories :: Religion & Spirituality , Business, Education, Comedy, Health & Fitness')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()


In [ ]:
merged = pd.merge(df_podcasts,df_users, left_on = 'id', right_on = 'podcasts_id', how = 'inner')
merged = merged[['podcasts_id','name','url','studio','category','episode_count','avg_rating','total_ratings','description', 
       'username','review_title','review','rating','date']]
df_users_ratings = merged[['podcasts_id','name','username','category','rating']]

In [ ]:

import plotly.express as px
df = df_users_ratings
fig = px.histogram(df, x="rating",nbins=10,color='rating',
                   labels={'episode_count':'No of episodes in a podcast'})
fig.update_layout(title='Distribution of ratings by users')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")

In [ ]:
df_top_10_rated_podcasts = pd.DataFrame(df_users_ratings.groupby(by='name')['rating'].count().sort_values(ascending=False)[0:10]).reset_index()
df_top_10_rated_podcasts.rename(columns={"rating": "no of ratings per podcast"},inplace=True)

In [ ]:
df_top_10_rated_podcasts

In [ ]:
df = df_top_10_rated_podcasts
fig = px.bar(df, x='name', y='no of ratings per podcast', color='no of ratings per podcast',)
fig.update_layout(title='Top 10 Podcasts based on number of ratings')
fig.update_yaxes(tickfont_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black")
fig.show()

In [ ]:
df_podcasts.to_csv(PODCASTS_DATABASE_PATH_PROCESSED + '/' + "df_podcasts.csv")
df_users.to_csv(USER_REVIEWS_DATABASE_PATH_PROCESSED + '/' + "df_users.csv")